In [1]:
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance,PointStruct
from img2vec_pytorch import Img2Vec
import glob
import os
import csv
from PIL import Image

In [2]:
client = QdrantClient(url="http://localhost:6333")
model = Img2Vec(model='efficientnet_b0')

/Users/lucas/Developer/tcc/venv/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/lucas/Developer/tcc/venv/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [3]:
def generate_embedding(filename):
    img = Image.open(filename).convert("RGB")
    vec = model.get_vec(img=img)
    return vec.tolist()

In [4]:
with open("/Users/lucas/Developer/tcc/datasets/full/sb_test.csv", 'r', encoding='utf-8') as all_images:
    reader = csv.DictReader(all_images)
    lines = list(reader)

In [5]:
dir_name = "/Users/lucas/Developer/tcc/datasets/full/images_jpg"
image_glob = glob.glob(dir_name + '/*' + '.jpeg')
id = 1

In [6]:
def search(vector):
    hits = client.search(
        collection_name="building-patterns",
        query_vector=vector,
        limit=1 
    )
    return hits


In [7]:
acertos = 0
erros = 0
total_test = len(lines)
print(total_test)

285


In [8]:
for image in image_glob:
    for line in lines:
        try:
            if str(os.path.basename(image)).replace('.jpeg','') in str(line['foto']):
                vector = generate_embedding(image)
                hits = search(vector)
                if hits[0].payload['building_pattern'] == line['padrao_construtivo']:
                    acertos += 1
                else:
                    erros += 1
                print("Hit:", hits)
        except Exception as e:
            print(str(e))

Hit: [ScoredPoint(id=563, version=11, score=0.80017567, payload={'building_pattern': 'Médio Baixo', 'class_id': '4'}, vector=None, shard_key=None)]
Hit: [ScoredPoint(id=563, version=11, score=0.60952926, payload={'building_pattern': 'Médio Baixo', 'class_id': '4'}, vector=None, shard_key=None)]
Hit: [ScoredPoint(id=784, version=15, score=0.8435756, payload={'building_pattern': 'Médio Baixo', 'class_id': '4'}, vector=None, shard_key=None)]
Hit: [ScoredPoint(id=117, version=2, score=0.74084276, payload={'building_pattern': 'Médio', 'class_id': '3'}, vector=None, shard_key=None)]
Hit: [ScoredPoint(id=26, version=0, score=0.61181384, payload={'building_pattern': 'Médio Baixo', 'class_id': '4'}, vector=None, shard_key=None)]
Hit: [ScoredPoint(id=172, version=3, score=0.42811966, payload={'building_pattern': 'Médio', 'class_id': '3'}, vector=None, shard_key=None)]
Hit: [ScoredPoint(id=729, version=14, score=0.73006344, payload={'building_pattern': 'Médio', 'class_id': '3'}, vector=None, shar

In [9]:
accuracy = acertos/total_test
print('Acurácia =', accuracy)
print("Total de acertos = ", acertos)
print("Total de erros = ", erros)
print("Soma acertos + error = ", acertos +erros)
print("Total dataset = ", total_test)

Acurácia = 0.5052631578947369
Total de acertos =  144
Total de erros =  142
Soma acertos + error =  286
Total dataset =  285
